<h1> Question 1 - Importation of tweeets</h1>

In [40]:
import GetOldTweets3 as got
import pandas as pd
import numpy as np
import csv
from datetime import date, timedelta

#API for text analysis https://developer.aylien.com
from aylienapiclient import textapi

In [50]:
with open("data/trump.csv", 'w', encoding="utf-8") as f:
    # Wrap the file object by the writer.
    writer = csv.writer(f)
    
    # Write the header line to the output if needed.
    writer.writerow(['date', 'tweet'])
    
    start = date(2019, 5, 1)
    end = date(2019, 5, 3)
    #end = date.today()
    day = timedelta(days=1)

    mydate = start
    
    while mydate < end:
        print("{date.day:02}{date.month:02}{date.year}".format(date=mydate))
        
        tweetCriteria = got.manager.TweetCriteria().setUsername("realDonaldTrump")\
                                           .setSince(("{date.year}-{date.month:02}-{date.day:02}".format(date=mydate)))\
                                           .setUntil(("{date.year}-{date.month:02}-{date.day:02}".format(date=mydate+day)))\
                                           .setTopTweets(True)\
                                           .setMaxTweets(240)
        
        mydate = mydate + day
        
    
        # Loop through the dictionary and get the tuples.
        # You may want to sort it somehow. Not shown here.
        for tweets in got.manager.TweetManager.getTweets(tweetCriteria):
            #writer.writerow(t)

            writer.writerow([tweets.date, tweets.text])

01052019
02052019


In [84]:
df = pd.read_csv("data/trump.csv")
df.head()

,date,tweet
0,2019-05-01 13:03:41+00:00,"NO COLLUSION, NO OBSTRUCTION. Besides, how can..."
1,2019-05-01 12:28:53+00:00,Why didn’t President Obama do something about ...
2,2019-05-01 12:06:10+00:00,"Congress must change the Immigration Laws now,..."
3,2019-05-01 11:11:42+00:00,“No President in history has endured such vici...
4,2019-05-01 11:01:15+00:00,Gallup Poll: 56% of Americans rate their finan...


In [12]:
#Test of the API
from aylienapiclient import textapi
client = textapi.Client("f4b7ab05", "bd00d1ce1dffc51dff21d2bfbeef6f37")
s = c.Sentiment({'text': 'The tree is green'})
s

{'polarity': 'neutral',
 'subjectivity': 'subjective',
 'text': 'The tree is green',
 'polarity_confidence': 0.6618327498435974,
 'subjectivity_confidence': 0.9879279463194073}

In [8]:
s = c.Sentiment({'text': 'John is shit, dumb and big fat ass'})
s

{'polarity': 'negative',
 'subjectivity': 'subjective',
 'text': 'John is shit, dumb and big fat ass',
 'polarity_confidence': 0.987261176109314,
 'subjectivity_confidence': 0.999989838019159}

<h1> Question 2 - Creation of a daily sentiment time serie</h1>

In [45]:
#Calls of the API are limited for freee users
client = textapi.Client("f4b7ab05", "bd00d1ce1dffc51dff21d2bfbeef6f37")

In [85]:
#We call the Sentiment API to analyse tweets
#It returns one polarity value for each tweet that can be  'positive', 'negative' or 'neutral'
df['polarity'] = client.Sentiment({'text': df['tweet']})['polarity']

In [86]:
#We convert polarity in -1,0 and 1 values
df['sentiment'] = (np.select(
        condlist=[df['polarity'] == 'positive', df['polarity'] == 'negative'], 
        choicelist=[1, -1], 
        default= 0))
df.head()

,date,tweet,polarity,sentiment
0,2019-05-01 13:03:41+00:00,"NO COLLUSION, NO OBSTRUCTION. Besides, how can...",negative,-1
1,2019-05-01 12:28:53+00:00,Why didn’t President Obama do something about ...,negative,-1
2,2019-05-01 12:06:10+00:00,"Congress must change the Immigration Laws now,...",negative,-1
3,2019-05-01 11:11:42+00:00,“No President in history has endured such vici...,negative,-1
4,2019-05-01 11:01:15+00:00,Gallup Poll: 56% of Americans rate their finan...,negative,-1


In [87]:
df['date'] = df['date'].apply(lambda x: x[:10])
df

,date,tweet,polarity,sentiment
0,2019-05-01,"NO COLLUSION, NO OBSTRUCTION. Besides, how can...",negative,-1
1,2019-05-01,Why didn’t President Obama do something about ...,negative,-1
2,2019-05-01,"Congress must change the Immigration Laws now,...",negative,-1
3,2019-05-01,“No President in history has endured such vici...,negative,-1
4,2019-05-01,Gallup Poll: 56% of Americans rate their finan...,negative,-1
5,2019-05-01,I am overriding the Decommission Order of the ...,negative,-1
6,2019-05-01,“The Democrats can’t come to grips with the fa...,negative,-1
7,2019-05-01,I’ve done more for Firefighters than this dues...,negative,-1
8,2019-05-02,Congrats to @U_S_Steel for investing $1+ BILLI...,negative,-1
9,2019-05-02,"On this day of prayer, we once again place our...",negative,-1


In [88]:
df.drop(['tweet', 'polarity'], axis='columns', inplace=True)
df

,date,sentiment
0,2019-05-01,-1
1,2019-05-01,-1
2,2019-05-01,-1
3,2019-05-01,-1
4,2019-05-01,-1


In [93]:
#We group tweets by date and do the mean for each day to get a time serie
df.groupby('date').mean()

,sentiment
date,
2019-05-01,-1
2019-05-02,-1


In [94]:
df

,date,sentiment
0,2019-05-01,-1
1,2019-05-01,-1
2,2019-05-01,-1
3,2019-05-01,-1
4,2019-05-01,-1
5,2019-05-01,-1
6,2019-05-01,-1
7,2019-05-01,-1
8,2019-05-02,-1
9,2019-05-02,-1


<h1> Question 3 - Properties of the sentiment index time series</h1>

<h1> Question 4 - Comparison bewtween tweets and S&P 500</h1>

<h1> Question 5 - Test for Granger causality</h1>